In [73]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import xlwt
from scipy import integrate
from skimage import measure,data,color


def cal_area(src):
    src[src > 0] = 1
    src_area = np.sum(src)
    return src_area

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):
    """Detect peaks in data based on their amplitude and other features.
    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height.
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).
    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.
    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`

    The function can handle NaN's
    See this IPython Notebook [1]_.
    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    Examples
    --------

    """

    x = np.atleast_1d(x).astype('float64')   # 将输入的数据转换成1维数组，浮点型
    if x.size < 3:          # 如果数组尺寸小于3，则返回空，表示没有波峰或波谷
        return np.array([], dtype=int)
    if valley:      # 表示当求波谷时，先将所有数据取反
        x = -x
    # find indexes of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan - 1, indnan + 1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size - 1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind] - x[ind - 1], x[ind] - x[ind + 1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                       & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indexes by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind

def rotation2(src):
    cnt = get_cnt(src)
    rect = cv2.minAreaRect(cnt)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = rect[2]

    if width < height:  # calculate the angle, prepare for next step
        angle = angle - 90

    src_pts = cv2.boxPoints(rect) # 使用cv2.boxPoints()可获取该矩形的四个顶点坐标
    src_pts = src_pts.astype('float32')
    dst_pts = np.array([[0, height],
                        [0, 0],
                        [width, 0],
                        [width, height]], dtype="float32")
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)   # 返回由源图像中矩形到目标图像矩形变换的矩阵，src：源图像中待测矩形的四点坐标，sdt：目标图像中矩形的四点坐标
    warped = cv2.warpPerspective(src, matrix, (width, height)) # 参数为：src，输入图像；M：变换矩阵；dsize：目标图像shape

    if angle <= -90:  # rotate for -90 degrees images    # 将图像旋转90度
        warped = cv2.transpose(warped)  # 转置图像
        warped = cv2.flip(warped, 1)  # anticlockwise 90 degrees, if u want clockwise, turn 0 to 1逆时针旋转90度，如果您想顺时针旋转，则将0旋转至1
        warped = np.rot90(warped, k=3) #将矩阵A逆时针旋转（90×k）°以后返回B，k取负数时表示顺时针旋转。
    else:
        warped = np.rot90(warped, k=1)
    return warped

def get_cnt(src):
    #gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY)
    if np.sum(binary) == 0:
        cnt = np.array([[[0,0]]])
    else:
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if contours == []:
            cnt = np.array([[[0,0]]])
        else:
            cnt = contours[0]
    return cnt

def minmax(area):
    maxi = max(area)
    mini = min(area)
    return mini,maxi

def fill_contour(img):
    contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    n = len(contours)  # 轮廓的个数
    max_area = 0
    for i  in range(n):
        if cv2.contourArea(contours[i]) > max_area:
            max_area = cv2.contourArea(contours[i])
    cv_contours = []
    if n != 1:
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < max_area:
                cv_contours.append(contour)
                x, y, w, h = cv2.boundingRect(contour)
                img[y:y + h, x:x + w] = 1
            else:
                continue
    else:
        pass
    
    return img
def roi_dect(cav_npy):
    frame = cav_npy[:,:,:]
    frame  = np.concatenate((frame , frame , frame ), axis=-1)
    #frame = cv2.resize(frame, (shape0,shape1))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #plt.imshow(frame)
    #print(frame2.shape)
    frame = np.array(frame,np.uint8)
    frame_detect = rotation2(frame)
    #plt.figure()
    #plt.imshow(frame_detect)
    return frame_detect

def lvd_dect(cav_npy):
    frame = cav_npy[:, :]
    frame  = np.concatenate((frame , frame , frame ), axis=-1)
    #print(frame.shape)
    #frame = cv2.resize(frame, (shape0,shape1))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #plt.imshow(frame)
    #print(frame2.shape)
    frame = np.array(frame,np.uint8)
    frame_detect = rotation2(frame)
    #plt.figure()
    #plt.imshow(frame_detect)
    return frame_detect

def cal_keyframe(srcnpy,i):
    #srcnpy = np.load(srcnpy_path)[:, :, :, :, np.newaxis]
    #for i in range(srcnpy.shape[0]):
    area = []
    lvd = []
    volume = []
    for k in range(srcnpy.shape[1]):
        area1 = cal_area(srcnpy[i,k,:,:,:])
        area.append(area1)
        img = srcnpy[i,k,:,:,:]
        img = np.array(img,np.uint8)
        img = fill_contour(img)
        l1 = int(max(img.sum(axis=0)))
        #print(l1)
        #lvd1 = lvd_dect(img).shape[0]
        lvd1 = l1
        lvd.append(lvd1)
        volume1 = 2/1000*0.85*(area1**2)/lvd1
        volume.append(volume1)
    area = np.array(area)
    area_avg = np.mean(area)
    peak_h = detect_peaks(area,mph=np.mean(area), mpd=len(area)/5)
    peak_l = detect_peaks(area, mpd=len(area)/4, valley=True, threshold=-(2*np.mean(area))/3)
    minvol,maxvol = minmax(volume)
    ef = (maxvol-minvol)/maxvol
    peak_list = np.zeros(srcnpy.shape[1])
    #print(peak_h)
    for i in range(len(peak_h)):
        peak_list[peak_h[i]] = 1
    for i in range(len(peak_l)):
        peak_list[peak_l[i]] = -1
    #print(peak_list)
    return area,volume,peak_h,peak_l,peak_list,lvd,minvol,maxvol,ef

def plot_peaks(x, indexes, algorithm=None, mph=None, mpd=None):
    """Plot results of the peak dectection."""
    try:
        import matplotlib.pyplot as plt
    except ImportError as e:
        print('matplotlib is not available.', e)
        return
    _, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.plot(x, 'b', lw=1)
    if indexes.size:
        label = 'peak'
        label = label + 's' if indexes.size > 1 else label
        ax.plot(indexes, x[indexes], '+', mfc=None, mec='r', mew=2, ms=8,
                label='%d %s' % (indexes.size, label))
        ax.legend(loc='best', framealpha=.5, numpoints=1)
    ax.set_xlim(-.02*x.size, x.size*1.02-1)
    ymin, ymax = x[np.isfinite(x)].min(), x[np.isfinite(x)].max()
    yrange = ymax - ymin if ymax > ymin else 1
    # ax.set_ylim(ymin - 0.1*yrange, ymax + 0.1*yrange)
#     ax.set_xlabel('frame', fontsize=14)
#     ax.set_ylabel('area', fontsize=14)
    # ax.set_title('%s (mph=%s, mpd=%s)' % (algorithm, mph, mpd))
    plt.show()

In [74]:
road = '/data/zhangzhenxuan/zhangzhenxuan/zhangzhenxuan'

In [121]:
# path1 = road+'/MTL_LOSS/multi-task/nature_1_pred.npy'
# prdnpy = np.load(path1).reshape(1990, 30, 128, 128, 1)
path1 = road+'/MTL_LOSS/multi-task/nature_2_pred.npy'
prdnpy = np.load(path1).reshape(1990, 30, 128, 128, 1)
path2 = '/data/zhangzhenxuan/nature_data/gts_a4c_1.npy'
srcnpy = np.load(path2)[:, :, :, :, np.newaxis]
print(prdnpy.shape,srcnpy.shape)

(1990, 30, 128, 128, 1) (1990, 30, 128, 128, 1)


In [ ]:
peak_list_all = []
ef_pr = []
ef_gt = []
edv_pr = []
edv_gt = []
esv_pr = []
esv_gt = []
area_pr = []
area_gt = []
vol_pr = []
vol_gt = []
for i in range(0,1000):
    area1,vol1,peak_h1,peak_l1,peak_list1,lvd1,minvol1,maxvol1,ef1 = cal_keyframe(prdnpy,i)
    area2,vol2,peak_h2,peak_l2,peak_list2,lvd2,minvol2,maxvol2,ef2 = cal_keyframe(srcnpy,i)
    print(ef1,ef2)
    print(minvol1,maxvol1,minvol2,maxvol2)
    ef_pr.append(ef1)
    ef_gt.append(ef2)
    edv_pr.append(maxvol1)
    edv_gt.append(maxvol2)
    esv_pr.append(minvol1)
    esv_gt.append(minvol2)
    area_pr.append(area1)
    area_gt.append(area2)
    vol_pr.append(vol1)
    vol_gt.append(vol2)

In [ ]:
vol_pr = np.array(vol_pr)
vol_gt = np.array(vol_gt)
area_pr = np.array(area_pr)/100
area_gt = np.array(area_gt)/100
print(vol_pr.shape,vol_gt.shape)
print(vol_gt.mean(axis = 0))

In [ ]:
def bland(data1,data2, *args, **kwargs):
    data1=np.asarray(data1.reshape(-1))
    data2=np.asarray(data2.reshape(-1))
    #print(data1)
    mean=np.mean([data1,data2],axis = 0)
    diff=data1-data2
#     print(diff)
    md=np.mean(diff)
    zero = 0
    sd=np.std(diff,axis = 0)
#     print(sd)
    #plt.figure(figsize=(7, 7))
    plt.figure(dpi=300,figsize=(7, 7))
    # plt.scatter(mean,diff,marker='o',s=22,c='#E5E12F',edgecolor='#6F0707',*args,** kwargs)
    plt.scatter(mean,diff,marker='o',s=62,c='#273B66',edgecolor='#23A4E7',*args,** kwargs)
    plt.axhline(md,color = 'black',linewidth = 8,linestyle = '--',label='md')
    #plt.axhline(zero,color = 'gray',linestyle = '--',label='md')
    plt.axhline(md+1.96 * sd,color = 'crimson',linewidth = 8,linestyle = '-.',label='md+1.96*sd')
    plt.axhline(md-1.96 * sd,color = 'crimson',linewidth = 8,linestyle = '-.',label='md-1.96*sd')
    #plt.legend()
    plt.text(x=max(mean), y=(md+1.96 * sd)+sd*0.1,s=str(round(md+1.96 * sd,2)),fontsize=15)
    plt.text(x=max(mean), y=md+sd*0.1,s=str(round(md,2)),fontsize=15)
    plt.text(x=max(mean), y=(md-1.96 * sd)+sd*0.1,s=str(round(md-1.96 * sd,2)),fontsize=15)
    x = np.linspace(min(mean)-20,max(mean)+20,200)
    y1 = md+1.96 * sd * np.ones(200)
    y2 = md-1.96 * sd * np.ones(200)
    #plt.fill_between(x, y1,y2,color = 'crimson', alpha=0.05)
    #plt.ylabel('Difference between groundtruth ESV \n and predicted ESV',fontsize=15)
    #plt.xlabel('Average between groundtruth ESV and predicted ESV',fontsize=15)
    plt.ylim([-(1.96 * sd)*2,(1.96 * sd)*2])
    plt.xlim([min(mean)-(1/5)*max(mean),max(mean)+(1/5)*max(mean)])
    #plt.title('Bland-Altman Analyse',fontsize=25)
    bwith = 2 #边框宽度设置为2
    TK = plt.gca()#获取边框
    TK.spines['bottom'].set_linewidth(bwith)#图框下边
    TK.spines['left'].set_linewidth(bwith)#图框左边
    TK.spines['top'].set_linewidth(bwith)#图框上边
    TK.spines['right'].set_linewidth(bwith)#图框右边
    plt.savefig('BA-'+indic+'_deepblue.png', format='png', transparent=False, dpi=300)
    #plt.show()
    print(md+1.96 * sd,md-1.96 * sd,md)
indic = 'ef'
if indic == 'ef':
    bland(np.array(ef_pr),np.array(ef_gt))
elif indic == 'esv':
    bland(np.array(esv_pr),np.array(esv_gt))
elif indic == 'edv':
    bland(np.array(edv_pr),np.array(edv_gt))


In [ ]:
from scipy.stats import gaussian_kde
import matplotlib.colors as colors
def bland(data1,data2, *args, **kwargs):
    data1=np.asarray(data1.reshape(-1))
    data2=np.asarray(data2.reshape(-1))
    #print(data1)
    mean=np.mean([data1,data2],axis = 0)
    diff=data1-data2
#     print(diff)
    md=np.mean(diff)
    zero = 0
    sd=np.std(diff,axis = 0)
#     print(sd)
    #plt.figure(figsize=(7, 7))
    plt.figure(dpi=300,figsize=(9, 7))
    # plt.scatter(mean,diff,marker='o',s=22,c='#E5E12F',edgecolor='#6F0707',*args,** kwargs)
    # plt.scatter(mean,diff,marker='o',s=62,c='#273B66',edgecolor='#23A4E7',*args,** kwargs)
    xy = np.vstack([mean,diff])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    mean, diff, z = mean[idx], diff[idx], z[idx]
    # mean_m, diff_m = np.meshgrid(mean, diff)
    # z = [z, z]
    scatter = plt.scatter(mean,diff,marker='o',s=62,c=z**(1/8),cmap='PuBu',*args,** kwargs)
    # scatter = plt.pcolormesh(mean_m,diff_m, z, cmap ='blues', vmin = z_min, vmax = z_max)
    cbar=plt.colorbar(scatter,shrink=1,orientation='vertical',extend='both',pad=0.015,aspect=30)
    # plt.pcolormesh(mean,diff, z,norm=colors.SymLogNorm(linthresh=0.4, linscale=1,vmin=0, vmax=8, base=10))
    plt.axhline(md,color = 'black',linewidth = 8,linestyle = '--',label='md')
    #plt.axhline(zero,color = 'gray',linestyle = '--',label='md')
    plt.axhline(md+1.96 * sd,color = 'crimson',linewidth = 8,linestyle = '-.',label='md+1.96*sd')
    plt.axhline(md-1.96 * sd,color = 'crimson',linewidth = 8,linestyle = '-.',label='md-1.96*sd')
    #plt.legend()
    plt.text(x=max(mean), y=(md+1.96 * sd)+sd*0.1,s=str(round(md+1.96 * sd,2)),fontsize=15)
    plt.text(x=max(mean), y=md+sd*0.1,s=str(round(md,2)),fontsize=15)
    plt.text(x=max(mean), y=(md-1.96 * sd)+sd*0.1,s=str(round(md-1.96 * sd,2)),fontsize=15)
    x = np.linspace(min(mean)-20,max(mean)+20,200)
    y1 = md+1.96 * sd * np.ones(200)
    y2 = md-1.96 * sd * np.ones(200)
    #plt.fill_between(x, y1,y2,color = 'crimson', alpha=0.05)
    #plt.ylabel('Difference between groundtruth ESV \n and predicted ESV',fontsize=15)
    #plt.xlabel('Average between groundtruth ESV and predicted ESV',fontsize=15)
    plt.ylim([-(1.96 * sd)*2,(1.96 * sd)*2])
    plt.xlim([min(mean)-(1/5)*max(mean),max(mean)+(1/5)*max(mean)])
    #plt.title('Bland-Altman Analyse',fontsize=25)
    bwith = 2 #边框宽度设置为2
    TK = plt.gca()#获取边框
    TK.spines['bottom'].set_linewidth(bwith)#图框下边
    TK.spines['left'].set_linewidth(bwith)#图框左边
    TK.spines['top'].set_linewidth(bwith)#图框上边
    TK.spines['right'].set_linewidth(bwith)#图框右边
    plt.savefig('BA_gau-'+indic+'_deepblue.png', format='png', transparent=False, dpi=300)
    #plt.show()
    print(md+1.96 * sd,md-1.96 * sd,md)
indic = 'ef'
if indic == 'ef':
    bland(np.array(ef_pr),np.array(ef_gt))
elif indic == 'esv':
    bland(np.array(esv_pr),np.array(esv_gt))
elif indic == 'edv':
    bland(np.array(edv_pr),np.array(edv_gt))

In [14]:
import pandas as pd
data_frame=pd.read_excel('./ef.xls')
# indic = 'esv'
a_ef = data_frame[indic+'gt']
b_ef = data_frame[indic+'pr']
#c = data_frame['Bedside EF']
a_ef = np.array(a_ef)
b_ef = np.array(b_ef)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
if indic == 'ef':
    a,b = np.array(ef_pr)*100,np.array(ef_gt)*100
elif indic == 'esv':
    a,b = np.array(esv_pr),np.array(esv_gt)
elif indic == 'edv':
    a,b = np.array(edv_pr),np.array(edv_gt)
plt.figure(dpi=300,figsize=(7, 7))
plt.plot(np.arange(0, max(a)+(1/10)*max(a)),np.arange(0, max(a)+(1/10)*max(a)),color = 'k',linewidth = 8,linestyle = '-.')
plt.scatter(abs(a),abs(b),marker='o',s=62,c='#273B66',edgecolor='#23A4E7')
# plt.scatter(abs(a_ef),abs(b_ef),marker='o',s=18,c='magenta',edgecolor='b')
plt.scatter(abs(a_ef),abs(b_ef),marker='o',s=62,c='#273B66',edgecolor='#23A4E7')
y_est = a
y_err = a.std() * np.sqrt(1/len(a) +(a - a.mean())**2 / np.sum((a - a.mean())**2))
y_est = np.array(np.sort(y_est))
y_err = np.array(np.sort(y_err))
plt.ylim([0,100])
plt.xlim([0,100])
bwith = 2 #边框宽度设置为2
TK = plt.gca()#获取边框
TK.spines['bottom'].set_linewidth(bwith)#图框下边
TK.spines['left'].set_linewidth(bwith)#图框左边
TK.spines['top'].set_linewidth(bwith)#图框上边
TK.spines['right'].set_linewidth(bwith)#图框右边
plt.savefig('cor'+indic+'_deepblue.png', format='png', transparent=False, dpi=300)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
indic = 'ef'
if indic == 'ef':
    a,b = np.array(ef_pr)*100,np.array(ef_gt)*100
elif indic == 'esv':
    a,b = np.array(esv_pr),np.array(esv_gt)
elif indic == 'edv':
    a,b = np.array(edv_pr),np.array(edv_gt)
plt.figure(dpi=300,figsize=(9, 7))
plt.plot(np.arange(0, max(a)+(1/10)*max(a)),np.arange(0, max(a)+(1/10)*max(a)),color = 'k',linewidth = 8,linestyle = '-.')
# plt.scatter(abs(a),abs(b),marker='o',s=62,c='#273B66',edgecolor='#23A4E7')
# plt.scatter(abs(a_ef),abs(b_ef),marker='o',s=62,c='#273B66',edgecolor='#23A4E7')
a = np.concatenate((a,a_ef),axis=0)
b = np.concatenate((b,b_ef),axis=0)
xy = np.vstack([a,b])
z = gaussian_kde(xy)(xy)
idx = z.argsort()
a,b, z = a[idx], b[idx], z[idx]
# mean_m, diff_m = np.meshgrid(mean, diff)
# z = [z, z]
scatter = plt.scatter(a,b,marker='o',s=62,c=z**(1/8),cmap='PuBu')
# scatter = plt.pcolormesh(mean_m,diff_m, z, cmap ='blues', vmin = z_min, vmax = z_max)
cbar=plt.colorbar(scatter,shrink=1,orientation='vertical',extend='both',pad=0.015,aspect=30)
y_est = a
y_err = a.std() * np.sqrt(1/len(a) +(a - a.mean())**2 / np.sum((a - a.mean())**2))
y_est = np.array(np.sort(y_est))
y_err = np.array(np.sort(y_err))
plt.ylim([0,100])
plt.xlim([0,100])
bwith = 2 #边框宽度设置为2
TK = plt.gca()#获取边框
TK.spines['bottom'].set_linewidth(bwith)#图框下边
TK.spines['left'].set_linewidth(bwith)#图框左边
TK.spines['top'].set_linewidth(bwith)#图框上边
TK.spines['right'].set_linewidth(bwith)#图框右边
#print(y_est.shape,y_err.shape)
#plt.plot(y_est,y_est + y_err,color = 'crimson',linewidth = 1,linestyle = '-.')
#plt.plot(y_est,y_est - y_err,color = 'crimson',linewidth = 1,linestyle = '-.')
#plt.fill_between(y_est, y_est - y_err, y_est + y_err, color = 'crimson', alpha=0.05)
#plt.ylabel('Predicted ESV',fontsize=15)
#plt.xlabel('GroundTruth ESV',fontsize=15)
#plt.title('Correlation Analyse',fontsize=25)
plt.savefig('cor_gau'+indic+'_deepblue.png', format='png', transparent=False, dpi=300)